In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd

ox.config(log_console=True, use_cache=True)
weight_by_length = False

In [2]:
df = pd.read_csv('data/ACS_17_5YR_B01003_with_ann.csv', dtype={'Id2':str})
df = df.rename(columns={'Estimate; Total':'pop_total'})
len(df)

739

In [3]:
gdf = gpd.read_file('data/tl_2018_37_place', dtype={'GEOID':str})
len(gdf)

739

In [4]:
places_all = pd.merge(gdf, df, how='inner', left_on='GEOID', right_on='Id2')
len(places_all)

739

In [5]:
places = places_all.sort_values('pop_total', ascending=False).head(25).reset_index()[['NAME', 'pop_total', 'geometry']]
len(places)

25

## Get the street networks and their edge bearings

In [6]:
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

In [7]:
bearings = {}
for label, row in places.iterrows():
    
    geometry = row['geometry']
    place = row['NAME']
    
    # get the graph
    G = ox.graph_from_polygon(geometry, network_type='drive')
    
    # calculate edge bearings
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    
    # don't weight bearings, just take one value per street segment
    b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
    bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')

## Visualize it

In [8]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = np.arange(n + 1) * 360 / n
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

In [9]:
# function to draw a polar histogram for a set of edge bearings
def polar_plot(ax, bearings, n=36, title=''):

    bins = np.arange(n + 1) * 360 / n
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width =  2 * np.pi / n

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')

    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
                  color='#003366', edgecolor='k', linewidth=0.5, alpha=0.7)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family':'Arial', 'size':24, 'weight':'bold'}
    xtick_font = {'family':'Arial', 'size':10, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family':'Arial', 'size': 9, 'weight':'bold', 'alpha':0.2, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [11]:
# create figure and axes
n = len(places)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})

# plot each city's polar histogram
for ax, place in zip(axes.flat, places['NAME'].sort_values().values):
    polar_plot(ax, bearings[place].dropna(), title=place)

# add super title and save full image
suptitle_font = {'family':'Arial', 'fontsize':60, 'fontweight':'normal', 'y':1.07}
fig.suptitle('City Street Network Orientation', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
fig.savefig('street-orientations.png', dpi=120, bbox_inches='tight')
plt.close()

In [12]:
places

,NAME,pop_total,geometry
0,Charlotte,826060,POLYGON ((-81.00953899999999 35.15149299999999...
1,Raleigh,449477,"(POLYGON ((-78.685496 35.742721, -78.685468 35..."
2,Greensboro,284816,"(POLYGON ((-79.627657 36.056352, -79.626458 36..."
3,Durham,257232,"(POLYGON ((-78.767985 35.923513, -78.767964999..."
4,Winston-Salem,240193,"(POLYGON ((-80.29413099999999 36.006652, -80.2..."
5,Fayetteville,210324,"(POLYGON ((-78.778561 35.040711, -78.778471999..."
6,Cary,159715,"(POLYGON ((-78.741929 35.73363, -78.741271 35...."
7,Wilmington,115261,"(POLYGON ((-77.80510000000001 34.286427, -77.8..."
8,High Point,109849,"(POLYGON ((-79.924403 35.96845, -79.922966 35...."
9,Greenville,90347,"(POLYGON ((-77.31837999999999 35.511695, -77.3..."
